In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# CONFIGURACIÓN
# Cambia 'postgres' si tu usuario es diferente
# Cambia 'TU_CONTRASEÑA_AQUI' por tu contraseña real de PostgreSQL
USER = "postgres"
PASSWORD = "GGmp200403" 
HOST = "localhost"
PORT = "5432"
DB = "pronto_lab"

# CREAR CONEXIÓN
connection_str = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}"
engine = create_engine(connection_str)

# PROBAR CONEXIÓN
try:
    with engine.connect() as connection:
        print("✅ ¡Éxito! Conexión OK a la base de datos:", DB)
except Exception as e:
    print("❌ Error de conexión. Revisa tu contraseña o si creaste la base de datos.")
    print(e)

Creacion de tablas

In [ ]:
from sqlalchemy import text

# Definición del esquema y tablas (DDL)
ddl_script = """
CREATE SCHEMA IF NOT EXISTS etl_project;

-- 1. Tabla de ESTACIONES (Stations)
DROP TABLE IF EXISTS etl_project.stations CASCADE;
CREATE TABLE etl_project.stations (
    station_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(150),
    lat NUMERIC,
    long NUMERIC,
    install_date DATE,
    install_dockcount INT,
    modification_date DATE,
    current_dockcount INT,
    decommission_date DATE
);

-- 2. Tabla de CLIMA (Weather)
DROP TABLE IF EXISTS etl_project.weather CASCADE;
CREATE TABLE etl_project.weather (
    date DATE PRIMARY KEY,
    max_temperature_f INT,
    mean_temperature_f INT,
    min_temperature_f INT,
    max_dew_point_f INT,
    mean_dew_point_f INT,
    min_dew_point_f INT,
    max_humidity INT,
    mean_humidity INT,
    min_humidity INT,
    max_sea_level_pressure_in NUMERIC,
    mean_sea_level_pressure_in NUMERIC,
    min_sea_level_pressure_in NUMERIC,
    max_visibility_miles NUMERIC,
    mean_visibility_miles NUMERIC,
    min_visibility_miles NUMERIC,
    max_wind_speed_mph INT,
    mean_wind_speed_mph INT,
    max_gust_speed_mph INT,
    precipitation_in NUMERIC,
    events VARCHAR(100)
);

-- 3. Tabla de VIAJES (Trips)
DROP TABLE IF EXISTS etl_project.trips CASCADE;
CREATE TABLE etl_project.trips (
    trip_id INT PRIMARY KEY,
    starttime TIMESTAMP,
    stoptime TIMESTAMP,
    bikeid VARCHAR(50),
    tripduration NUMERIC,
    from_station_id VARCHAR(50),
    from_station_name VARCHAR(150),
    to_station_id VARCHAR(50),
    to_station_name VARCHAR(150),
    usertype VARCHAR(50),
    gender VARCHAR(20),
    birthyear NUMERIC,
    -- Llaves foráneas que conectan viajes con estaciones
    FOREIGN KEY (from_station_id) REFERENCES etl_project.stations(station_id),
    FOREIGN KEY (to_station_id) REFERENCES etl_project.stations(station_id)
);
"""

# Ejecutar el código SQL
with engine.connect() as conn:
    conn.execute(text(ddl_script))
    conn.commit()
    print("✅ Tablas 'stations', 'weather' y 'trips' creadas exitosamente.")

Código de Carga de datos

In [ ]:
import pandas as pd
import numpy as np
import time
from sqlalchemy import text

# RUTAS
file_stations = 'data/station.csv'
file_weather = 'data/weather.csv'
file_trips = 'data/trip.csv'

# --- 1. CARGAR STATIONS ---
def cargar_stations():
    print("📂 Procesando STATIONS...")
    df = pd.read_csv(file_stations)
    cols_fecha = ['install_date', 'modification_date', 'decommission_date']
    for col in cols_fecha:
        df[col] = pd.to_datetime(df[col], format='mixed', errors='coerce')
    df.to_sql('stations', con=engine, schema='etl_project', if_exists='append', index=False)
    print("✅ Stations cargada.")

# --- 2. CARGAR WEATHER ---
def cargar_weather():
    print("📂 Procesando WEATHER...")
    df = pd.read_csv(file_weather)
    if 'Date' in df.columns:
        df = df.rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    
    mapa_columnas = {
        'Max_Temperature_F': 'max_temperature_f',
        'Mean_Temperature_F': 'mean_temperature_f',
        'Min_TemperatureF': 'min_temperature_f',
        'Max_Dew_Point_F': 'max_dew_point_f',
        'MeanDew_Point_F': 'mean_dew_point_f',
        'Min_Dewpoint_F': 'min_dew_point_f',
        'Max_Humidity': 'max_humidity',
        'Mean_Humidity': 'mean_humidity',
        'Min_Humidity': 'min_humidity',
        'Max_Sea_Level_Pressure_In': 'max_sea_level_pressure_in',
        'Mean_Sea_Level_Pressure_In': 'mean_sea_level_pressure_in',
        'Min_Sea_Level_Pressure_In': 'min_sea_level_pressure_in',
        'Max_Visibility_Miles': 'max_visibility_miles',
        'Mean_Visibility_Miles': 'mean_visibility_miles',
        'Min_Visibility_Miles': 'min_visibility_miles',
        'Max_Wind_Speed_MPH': 'max_wind_speed_mph',
        'Mean_Wind_Speed_MPH': 'mean_wind_speed_mph',
        'Max_Gust_Speed_MPH': 'max_gust_speed_mph',
        'Precipitation_In': 'precipitation_in',
        'Events': 'events'
    }
    df = df.rename(columns=mapa_columnas)
    columnas_sql = list(mapa_columnas.values()) + ['date']
    df_clean = df[df.columns.intersection(columnas_sql)].copy()
    
    for col in df_clean.columns:
        if col != 'date' and col != 'events':
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    df_clean.to_sql('weather', con=engine, schema='etl_project', if_exists='append', index=False)
    print("✅ Weather cargada.")

# --- 3. CARGAR TRIPS (Versión Anti-Duplicados) ---
def cargar_trips_final():
    print("📂 Procesando TRIPS (Validando FKs y Duplicados)...")
    
    # a. Leer CSV
    df_trips = pd.read_csv(file_trips, on_bad_lines='skip', low_memory=False)
    
    # b. ELIMINAR DUPLICADOS DE ID (NUEVO)
    cant_inicial = len(df_trips)
    df_trips = df_trips.drop_duplicates(subset=['trip_id'], keep='first')
    cant_duplicados = cant_inicial - len(df_trips)
    if cant_duplicados > 0:
        print(f"   ⚠️ Se eliminaron {cant_duplicados} registros con trip_id duplicado.")
    
    # c. Filtrar por Estaciones Válidas (FK)
    with engine.connect() as conn:
        valid_ids = pd.read_sql("SELECT station_id FROM etl_project.stations", conn)['station_id'].tolist()
    
    df_clean = df_trips[
        df_trips['from_station_id'].isin(valid_ids) & 
        df_trips['to_station_id'].isin(valid_ids)
    ].copy()
    
    print(f"   ⚠️ Se eliminaron {len(df_trips) - len(df_clean)} viajes con estaciones desconocidas.")

    # d. Fechas y Carga
    df_clean['starttime'] = pd.to_datetime(df_clean['starttime'], format='mixed')
    df_clean['stoptime'] = pd.to_datetime(df_clean['stoptime'], format='mixed')
    
    start_time = time.time()
    df_clean.to_sql('trips', con=engine, schema='etl_project', if_exists='append', index=False)
    end_time = time.time()
    print(f"✅ Trips cargada exitosamente en {end_time - start_time:.2f} s.")

# --- EJECUCIÓN ---
try:
    with engine.connect() as conn:
        print("🧹 Limpiando tablas para reintentar...")
        conn.execute(text("TRUNCATE TABLE etl_project.trips, etl_project.weather, etl_project.stations CASCADE;"))
        conn.commit()

    cargar_stations()
    cargar_weather()
    cargar_trips_final()
    
    print("\n🚀 ¡TODO CARGADO CORRECTAMENTE!")
    
    # Verificación final
    with engine.connect() as conn:
        print("Stations:", conn.execute(text("SELECT COUNT(*) FROM etl_project.stations")).scalar())
        print("Weather:", conn.execute(text("SELECT COUNT(*) FROM etl_project.weather")).scalar())
        print("Trips:", conn.execute(text("SELECT COUNT(*) FROM etl_project.trips")).scalar())

except Exception as e:
    print(f"\n❌ Error: {e}")

Actualización de datos Upsert

In [ ]:
from sqlalchemy import text

# 1. Definimos la sentencia UPSERT
# Fíjate en la parte "ON CONFLICT (date) DO UPDATE..."
sql_upsert = text("""
INSERT INTO etl_project.weather (date, max_temperature_f, mean_temperature_f, events)
VALUES (:date, :max_temp, :mean_temp, :events)
ON CONFLICT (date) 
DO UPDATE SET 
    max_temperature_f = EXCLUDED.max_temperature_f,
    mean_temperature_f = EXCLUDED.mean_temperature_f,
    events = EXCLUDED.events;
""")

# 2. Datos de prueba (Usaremos una fecha futura para no afectar datos reales)
fecha_demo = '2030-01-01'

# CASO A: Insertar por primera vez (Temperatura 85°F, Soleado)
dato_1 = {'date': fecha_demo, 'max_temp': 85, 'mean_temp': 80, 'events': 'Sunny'}

# CASO B: El clima cambió, corregimos el dato (Temperatura 30°F, Nieve)
dato_2 = {'date': fecha_demo, 'max_temp': 30, 'mean_temp': 25, 'events': 'Snow'}

with engine.connect() as conn:
    # --- Ejecución 1 ---
    conn.execute(sql_upsert, dato_1)
    conn.commit()
    print("1️⃣ Primera ejecución: Se insertó el registro del 2030 (Soleado).")
    
    # Verificamos
    res1 = conn.execute(text(f"SELECT * FROM etl_project.weather WHERE date = '{fecha_demo}'")).fetchone()
    print(f"   Resultado DB: {res1.max_temperature_f}°F - {res1.events}")

    # --- Ejecución 2 (Upsert) ---
    print("\n🔄 Ejecutando de nuevo con datos cambiados (Nieve)...")
    conn.execute(sql_upsert, dato_2)
    conn.commit()
    print("2️⃣ Segunda ejecución: Se actualizó el registro existente (sin error de duplicado).")
    
    # Verificamos cambio
    res2 = conn.execute(text(f"SELECT * FROM etl_project.weather WHERE date = '{fecha_demo}'")).fetchone()
    print(f"   Resultado DB: {res2.max_temperature_f}°F - {res2.events}")

Código de Carga

In [ ]:
import time

# 1. La consulta "pesada"
# Busca viajes de un mes específico y cuenta cuántos salieron de cada estación
query_pesada = """
SELECT from_station_id, COUNT(*) as total_viajes
FROM etl_project.trips
WHERE starttime >= '2015-01-01' AND starttime < '2015-02-01'
GROUP BY from_station_id
ORDER BY total_viajes DESC;
"""

def medir_tiempo(mensaje):
    start = time.time()
    # Ejecutamos la consulta
    pd.read_sql(query_pesada, engine)
    end = time.time()
    duracion = end - start
    print(f"{mensaje}: {duracion:.5f} segundos")
    return duracion

print("--- 🏁 INICIANDO PRUEBA DE VELOCIDAD 🏁 ---")

# A. Medir SIN índices
# (Para estar seguros, borramos índices si existían de pruebas anteriores)
with engine.connect() as conn:
    conn.execute(text("DROP INDEX IF EXISTS etl_project.idx_starttime;"))
    conn.execute(text("DROP INDEX IF EXISTS etl_project.idx_station_id;"))
    conn.commit()

tiempo_sin = medir_tiempo("🐢 Tiempo SIN índices")

# B. Crear los índices
print("\n🛠️ Creando índices (optimizando)...")
with engine.connect() as conn:
    start_idx = time.time()
    conn.execute(text("CREATE INDEX idx_starttime ON etl_project.trips(starttime);"))
    conn.execute(text("CREATE INDEX idx_station_id ON etl_project.trips(from_station_id);"))
    conn.commit()
    print(f"   Índices creados en {time.time() - start_idx:.2f} s.")

# C. Medir CON índices
tiempo_con = medir_tiempo("⚡ Tiempo CON índices")

# D. Conclusión
mejora = tiempo_sin - tiempo_con
porcentaje = (mejora / tiempo_sin) * 100
print(f"\n📊 CONCLUSIÓN: La consulta fue {porcentaje:.1f}% más rápida.")